In [ ]:
%load_ext autoreload
%autoreload 2
from ProjectRoot import change_wd_to_project_root
change_wd_to_project_root()

In [ ]:
from typing import List
from typing import Union
from pathlib import Path
import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
sns.set()
import pandas as pd
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator
# from train_federated_hps_fullres import NAME_TO_NODE_ID
from federated import NAME_TO_NODE_ID

## TEST

In [ ]:
def get_tb_metrics(tb_logdir):
    tb_logdir /= 'tb_logs'
    test_metrics = {}
    for node_tb_logdir in tb_logdir.iterdir():
        node_name = [k for k, v in NAME_TO_NODE_ID.items() if v == node_tb_logdir.name]
        if len(node_name) == 0:
            node_name = 'heidelberg'
        else:
            node_name = node_name[0]
        test_metrics[node_name] = {}
        ea = EventAccumulator(str(node_tb_logdir))
        ea.Reload()
        for tb_scalar in ea.Tags()['scalars']:
            test_metrics[node_name][tb_scalar.replace('VALIDATION ON GLOBAL UPDATES/', '')] = [x.value for x in ea.Scalars(tb_scalar)]
            #print(np.array([[x.step, x.wall_time, x.value] for x in ea.Scalars(tb_scalar)]))
    return test_metrics

In [ ]:
METHODS = ['SU', 'OU', 'ST', 'OT', 'SFU', 'OFU', 'SFT', 'OFT', 'SKDU', 'OKDU', 'SKDT', 'OKDT']

In [ ]:
hps_paths = dict(
    heidelberg_unet='experiments/federated_test/local_nnunet_heidelberg',
    heidelberg_swinunetr='experiments/federated_test/local_swinunetr_heidelberg',
    munich_unet='experiments/federated_test/local_nnunet_munich',
    munich_swinunetr='experiments/federated_test/local_swinunetr_munich',
    muenster_unet='experiments/federated_test/local_nnunet_muenster',
    muenster_swinunetr='experiments/federated_test/local_swinunetr_muenster',
    goettingen_unet='experiments/federated_test/local_nnunet_goettingen',
    goettingen_swinunetr='experiments/federated_test/local_swinunetr_goettingen',
    hamburg_unet='experiments/federated_test/local_nnunet_hamburg',
    hamburg_swinunetr='experiments/federated_test/local_swinunetr_hamburg'
    greifswald_unet='experiments/federated_test/local_nnunet_greifswald',
    greifswald_swinunetr='experiments/federated_test/local_swinunetr_greifswald',
    federated_unet='experiments/federated_test/federated_nnunet',
    federated_swinunetr='experiments/federated_test/federated_swinunetr',
    kd_unet='experiments/federated/test-kd-nnunet-hps-finetune',
    kd_swinunetr='experiments/federated_test/kd_swinunetr'
)

In [ ]:
hps_metrics = {n: get_tb_metrics(Path(p)) for n, p in hps_paths.items()}

In [ ]:
pts_labels = ['RCC', 'LCC', 'ACC', 'RCA', 'LCA']
data = []
for train_loc_model, metrics in hps_metrics.items():
    train_loc, model = train_loc_model.split('_')
    for test_loc, test_loc_metrics in metrics.items():
        for i in range(len(test_loc_metrics['Loss'])):
            test_loc_data = {l: test_loc_metrics[f'{l}_mm'][i] for l in pts_labels}
            test_loc_data['train_location'] = train_loc
            test_loc_data['test_location'] = test_loc
            test_loc_data['model'] = model
            data.append(test_loc_data)
df = pd.DataFrame(data)
df

In [ ]:
def filter_locs(x):
    # model_strs = {'unet': 'UNet', 'swinunetr': 'SWIN_UNETR'}
    model_strs = {'unet': 'U', 'swinunetr': 'T'}
    if x.train_location == x.test_location:
        return f'S{model_strs[x.model]}'
    elif x.train_location == 'federated':
        if x.test_location in ['heidelberg', 'goettingen', 'munich']:
            return f'SF{model_strs[x.model]}'
        else:
            return f'OF{model_strs[x.model]}'
    elif x.train_location == 'kd':
        if x.test_location in ['heidelberg', 'goettingen', 'munich']:
            return f'SKD{model_strs[x.model]}'
        else:
            return f'OKD{model_strs[x.model]}'
    elif x.train_location != x.test_location:
        return f'O{model_strs[x.model]}'
    else:
        print(x.test_location, x.train_location, x.model)

df['method'] = df.apply(filter_locs, axis=1)

In [ ]:
flattened_data = []
for i, row in df.iterrows():
    for j, l in enumerate(pts_labels):
        k = 'distance' # _hps' if j < 3 else 'distance_ca'
        flattened_data.append({
            'train_location': row.train_location, 
            'test_location': row.test_location, 
            'model': row.model,
            'method': row.method,
            k: row[l]
        })
flattened_df = pd.DataFrame(flattened_data)

In [ ]:
df = flattened_df
df['Model'] = df.method.apply(lambda x: x[1:])
df['Eval Clients'] = df.method.apply(lambda x: x[0])

MODELS = {'U': 'UNet', 'T': 'SWIN-UNETR', 'FU': 'FedUNet', 'FT': 'FedSWIN-UNETR', 'KDU': 'FedKDUNet', 'KDT': 'FedKDSWIN-UNETR'}
EVAL_CLIENTS = {'S': 'Training Clients', 'O': 'Other Clients'}
df['Model'] = df['Model'].apply(lambda x: MODELS[x])
df['Eval Clients'] = df['Eval Clients'].apply(lambda x: EVAL_CLIENTS[x])

fig, ax = plt.subplots(figsize=(10,5))
sns.boxplot(
    data=df, 
    x='Model', 
    y='distance', 
    hue='Eval Clients',
    ax=ax,
    hue_order=['Training Clients', 'Other Clients']
)
ax.set_ylim([0,10])
ax.set_title(r'Hinge Points and Coronary Ostia (HPs and COs) $\downarrow$', fontsize=24)
ax.set_ylabel('d [mm]', fontsize=18)
ax.set_xlabel(None)
ax.legend(loc='upper right')
fig.tight_layout()
fig.savefig('notebooks/images2/eval_hinge_points.png', bbox_inches='tight')

In [ ]:
for m in METHODS:
    vals = flattened_df.loc[flattened_df.method == m].distance.to_numpy()
    print(f'{m}: ${np.mean(vals[vals<10]):.2f} \pm {np.std(vals[vals<10]):.2f}$')

In [ ]:
medians = flattened_df.groupby('method')['distance'].median()
q25 = flattened_df.groupby('method')['distance'].quantile(.25)
q75 = flattened_df.groupby('method')['distance'].quantile(.75)

In [ ]:
for method in order:
    m = np.round(medians[method],2)
    mq25 = np.round(q25[method],2)
    mq75 = np.round(q75[method],2)
    iqr = mq75 - mq25
    print(f"{method}  ${m}\,(\mathrm{{IQR}}:\,{iqr:.2f})\,\mathrm{{mm}}$")

In [ ]:
pts_labels = ['RCC', 'LCC', 'ACC', 'RCA', 'LCA']
data = []
for train_loc_model, metrics in hps_metrics.items():
    train_loc, model = train_loc_model.split('_')
    for test_loc, test_loc_metrics in metrics.items():
        for i in range(len(test_loc_metrics['Loss'])):
            test_loc_data = {l: test_loc_metrics[f'{l}_DICE'][i] for l in pts_labels}
            test_loc_data['train_location'] = train_loc
            test_loc_data['test_location'] = test_loc
            test_loc_data['model'] = model
            data.append(test_loc_data)
df = pd.DataFrame(data)
flattened_data = []
for i, row in df.iterrows():
    for j, l in enumerate(pts_labels):
        k = 'dice' # _hps' if j < 3 else 'distance_ca'
        flattened_data.append({
            'train_location': row.train_location, 
            'test_location': row.test_location, 
            'model': row.model,
            # 'method': row.method,
            k: row[l]
        })
df = pd.DataFrame(flattened_data)
df

In [ ]:
from scipy.stats import ttest_rel
def pval(df,t1,t2,m1,m2):
    d1 = df[(df.train_location==t1) & (df.model==m1)]#.dice
    d2 = df[(df.train_location==t2) & (df.model==m2)]#.dice
    test_locs = [t for t in d1.test_location.unique() if t in d2.test_location.unique()]
    d1 = d1[d1.test_location.isin(test_locs)]
    d2 = d2[d2.test_location.isin(test_locs)]
    return ttest_rel(d1.dice,d2.dice).pvalue

In [ ]:
p = pval(df,'heidelberg','federated','unet','unet')
print('Local UNet / Fed UNet: ', p)

p = pval(df,'heidelberg','kd','unet','unet')
print('Local UNet / KD UNet: ', p)

p = pval(df,'heidelberg','federated','swinunetr','swinunetr')
print('Local Swin / Fed Swin: ', p)

p = pval(df,'heidelberg','kd','swinunetr','swinunetr')
print('Local Swin / KD Swin: ', p)

p = pval(df,'federated','kd','swinunetr','swinunetr')
print('Fed Swin / KD Swin: ', p)

p = pval(df,'federated','kd','unet','swinunetr')
print('Fed UNet / KD Swin: ', p)

p = pval(df,'federated','kd','unet','unet')
print('Fed UNet / KD UNet: ', p)

p = pval(df,'federated','federated','unet','swinunetr')
print('Fed UNet / Fed Swin: ', p)

# MS

checkpoints:
HD_UNET: ./checkpoints/ms/heidelberg.pt
HD_SWINUNETR: 
MS_UNET: ./checkpoints/ms/muenster.pt
HH_SWINUNETR: 
FED_UNET: ./checkpoints/ms/heidelberg_muenster.pt
FED_SWINUNETR: 


In [ ]:
ms_paths = dict(
    heidelberg_unet='experiments/federated/local_nnunet_ms_heidelberg',
    heidelberg_swinunetr='experiments/federated/local_swinunetr_ms_heidelberg',
    muenster_unet='experiments/federated/local_nnunet_ms_muenster',
    muenster_swinunetr='experiments/federated/ms_swinunetr_muenster',
    federated_unet='experiments/federated/federated_nnunet_ms',
    federated_swinunetr='experiments/federated/ms_swinunetr_fed',
    kd_unet='experiments/federated/test-kd-nnunet-ms-finetune',
    kd_swinunetr='experiments/federated/ms_swinunetr_kd'
)

In [ ]:
ms_metrics = {n: get_tb_metrics(Path(p)) for n, p in ms_paths.items()}

In [ ]:
pts_labels = ['MS1', 'MS2']
data = []
for train_loc_model, metrics in ms_metrics.items():
    train_loc, model = train_loc_model.split('_')
    for test_loc, test_loc_metrics in metrics.items():
        for i in range(len(test_loc_metrics['Loss'])):
            test_loc_data = {l: test_loc_metrics[f'{l}_mm'][i] for l in pts_labels}
            test_loc_data['train_location'] = train_loc
            test_loc_data['test_location'] = test_loc
            test_loc_data['model'] = model
            data.append(test_loc_data)
df = pd.DataFrame(data)
df

In [ ]:
def filter_locs(x):
    # model_strs = {'unet': 'UNet', 'swinunetr': 'SWIN_UNETR'}
    model_strs = {'unet': 'U', 'swinunetr': 'T'}
    if x.train_location == x.test_location:
        return f'S{model_strs[x.model]}'
    elif x.train_location == 'federated':
        if (x.test_location in ['heidelberg', 'muenster'] and x.model == 'unet') or (x.test_location in ['heidelberg', 'hamburg'] and x.model == 'swinunetr'):
            return f'SF{model_strs[x.model]}'
        else:
            return f'OF{model_strs[x.model]}'
    elif x.train_location == 'kd':
        if x.test_location in ['heidelberg', 'muenster']:
            return f'SKD{model_strs[x.model]}'
        else:
            return f'OKD{model_strs[x.model]}'
    elif x.train_location != x.test_location:
        return f'O{model_strs[x.model]}'
    else:
        print(x.test_location, x.train_location, x.model)

df['method'] = df.apply(filter_locs, axis=1)

In [ ]:
flattened_data = []
for i, row in df[df.method != 'KDU'].iterrows():
    for j, l in enumerate(pts_labels):
        k = 'distance' # _hps' if j < 3 else 'distance_ca'
        flattened_data.append({
            'train_location': row.train_location, 
            'test_location': row.test_location, 
            'model': row.model,
            'method': row.method,
            k: row[l]
        })
flattened_df = pd.DataFrame(flattened_data)

In [ ]:
df = flattened_df
df['Model'] = df.method.apply(lambda x: x[1:])
df['Eval Clients'] = df.method.apply(lambda x: x[0])

MODELS = {'U': 'UNet', 'T': 'SWIN-UNETR', 'FU': 'FedUNet', 'FT': 'FedSWIN-UNETR', 'KDU': 'FedKDUNet', 'KDT': 'FedKDSWIN-UNETR'}
EVAL_CLIENTS = {'S': 'Training Clients', 'O': 'Other Clients'}
df['Model'] = df['Model'].apply(lambda x: MODELS[x])
df['Eval Clients'] = df['Eval Clients'].apply(lambda x: EVAL_CLIENTS[x])

fig, ax = plt.subplots(figsize=(10,5))
sns.boxplot(
    data=df, 
    x='Model', 
    y='distance', 
    hue='Eval Clients',
    ax=ax,
    hue_order=['Training Clients', 'Other Clients']
)
ax.set_ylim([0,10])
ax.set_title(r'Membranous Septum (MS) $\downarrow$', fontsize=24)
ax.set_ylabel('d [mm]', fontsize=18)
ax.set_xlabel(None)
fig.tight_layout()
fig.savefig('notebooks/images2/eval_membranous_septum.png', bbox_inches='tight')

In [ ]:
for m in METHODS:
    vals = flattened_df.loc[flattened_df.method == m].distance.to_numpy()
    print(f'{m}: ${np.mean(vals[vals<10]):.2f} \pm {np.std(vals[vals<10]):.2f}$')

In [ ]:
pts_labels = ['MS1', 'MS2']
data = []
for train_loc_model, metrics in ms_metrics.items():
    train_loc, model = train_loc_model.split('_')
    for test_loc, test_loc_metrics in metrics.items():
        for i in range(len(test_loc_metrics['Loss'])):
            test_loc_data = {l: test_loc_metrics[f'{l}_DICE'][i] for l in pts_labels}
            test_loc_data['train_location'] = train_loc
            test_loc_data['test_location'] = test_loc
            test_loc_data['model'] = model
            data.append(test_loc_data)
df = pd.DataFrame(data)
flattened_data = []
for i, row in df.iterrows():
    for j, l in enumerate(pts_labels):
        k = 'dice' # _hps' if j < 3 else 'distance_ca'
        flattened_data.append({
            'train_location': row.train_location, 
            'test_location': row.test_location, 
            'model': row.model,
            # 'method': row.method,
            k: row[l]
        })
df = pd.DataFrame(flattened_data)
df

p = pval(df,'heidelberg','federated','unet','unet')
print('Local UNet / Fed UNet: ', p)

p = pval(df,'heidelberg','kd','unet','unet')
print('Local UNet / KD UNet: ', p)

p = pval(df,'hamburg','federated','swinunetr','swinunetr')
print('Local Swin / Fed Swin: ', p)

p = pval(df,'hamburg','kd','swinunetr','swinunetr')
print('Local Swin / KD Swin: ', p)

p = pval(df,'federated','kd','swinunetr','swinunetr')
print('Fed Swin / KD Swin: ', p)

p = pval(df,'federated','kd','unet','swinunetr')
print('Fed UNet / KD Swin: ', p)

p = pval(df,'federated','kd','unet','unet')
print('Fed UNet / KD UNet: ', p)

p = pval(df,'federated','federated','unet','swinunetr')
print('Fed UNet / Fed Swin: ', p)

In [ ]:
medians = flattened_df.groupby('method')['distance'].median()
q25 = flattened_df.groupby('method')['distance'].quantile(.25)
q75 = flattened_df.groupby('method')['distance'].quantile(.75)

In [ ]:
for method in order:
    m = np.round(medians[method],2)
    mq25 = np.round(q25[method],2)
    mq75 = np.round(q75[method],2)
    iqr = mq75 - mq25
    print(f"{method}  ${m}\,(\mathrm{{IQR}}:\,{iqr:.2f})\,\mathrm{{mm}}$")

# Calc

checkpoints:
HD_UNET:
HD_SWINUNETR: 
M_UNET: 
FFM_UNET: 
FED_UNET:
FED_SWINUNETR:
KD_SWINUNETR: 

In [ ]:
calc_paths = dict(
    heidelberg_unet='experiments/federated/calc_local_hd_unet',
    heidelberg_swinunetr='experiments/federated/calc_swinunetr_heidelberg',
    munich_unet='experiments/federated/calc_local_m_unet',
    munich_swinunetr='experiments/federated/calc_local_m_swinunetr',
    federated_unet='experiments/federated/test-fed-nnunet-calc',
    federated_swinunetr='experiments/federated/test-fed-swin_unetr-calc',
    kd_unet='experiments/federated/test-kd-nnunet-calc-finetune',
    kd_swinunetr='experiments/federated/test-kd-swin_unetr-calc-finetune'
)

In [ ]:
calc_metrics = {n: get_tb_metrics(Path(p)) for n, p in calc_paths.items()}

In [ ]:
import torch

data2 = []
for ffm_calc_dice_path in Path('.').glob('*_calc_dice.pt'):
    print(ffm_calc_dice_path)
    fname = str(ffm_calc_dice_path)

    method = 'swinunetr' if 'swinunetr' in fname else 'unet'
    if 'kd' in fname:
        tl = 'kd'
    elif '_hd_' in fname:
        tl = 'heidelberg'
    elif '_m_' in fname:
        tl = 'munich'
    elif 'fed' in fname:
        tl = 'federated'
    calc_dice = torch.load(ffm_calc_dice_path)['calc_DICE']
    for d in calc_dice:
        data2.append({
            'train_location': tl,
            'test_location': 'frankfurt',
            'model': method,
            'Calc': d
        })
    # break
pd.DataFrame(data2)

In [ ]:
pts_labels = ['Calc']
data = []
for train_loc_model, metrics in calc_metrics.items():
    train_loc, model = train_loc_model.split('_')
    for test_loc, test_loc_metrics in metrics.items():
        for i in range(len(test_loc_metrics['Loss'])):
            test_loc_data = {l: test_loc_metrics[f'{l}_DICE'][i] for l in pts_labels}
            test_loc_data['train_location'] = train_loc
            test_loc_data['test_location'] = test_loc
            test_loc_data['model'] = model
            data.append(test_loc_data)
df = pd.DataFrame(data+data2)
df

In [ ]:
def filter_locs(x):
    # model_strs = {'unet': 'UNet', 'swinunetr': 'SWIN_UNETR'}
    model_strs = {'unet': 'U', 'swinunetr': 'T'}
    if x.train_location == x.test_location:
        return f'S{model_strs[x.model]}'
    elif x.train_location == 'federated':
        if x.test_location in ['heidelberg', 'munich']:
            return f'SF{model_strs[x.model]}'
        else:
            return f'OF{model_strs[x.model]}'
    elif x.train_location == 'kd':
        if x.test_location in ['heidelberg', 'munich']:
            return f'SKD{model_strs[x.model]}'
        else:
            return f'OKD{model_strs[x.model]}'
    elif x.train_location != x.test_location:
        return f'O{model_strs[x.model]}'
    else:
        print(x.test_location, x.train_location, x.model)

df['method'] = df.apply(filter_locs, axis=1)

In [ ]:
df['Model'] = df.method.apply(lambda x: x[1:])
df['Eval Clients'] = df.method.apply(lambda x: x[0])

MODELS = {'U': 'UNet', 'T': 'SWIN-UNETR', 'FU': 'FedUNet', 'FT': 'FedSWIN-UNETR', 'KDU': 'FedKDUNet', 'KDT': 'FedKDSWIN-UNETR'}
EVAL_CLIENTS = {'S': 'Training Clients', 'O': 'Other Clients'}
df['Model'] = df['Model'].apply(lambda x: MODELS[x])
df['Eval Clients'] = df['Eval Clients'].apply(lambda x: EVAL_CLIENTS[x])

fig, ax = plt.subplots(figsize=(10,5))
sns.boxplot(
    data=df, 
    x='Model', 
    y='Calc', 
    hue='Eval Clients',
    ax=ax,
    hue_order=['Training Clients', 'Other Clients']
)
ax.set_title(r'Calcification $\uparrow$', fontsize=24)
ax.set_ylabel('DICE', fontsize=18)
ax.set_xlabel(None)
fig.tight_layout()
fig.savefig('notebooks/images2/eval_calc.png', bbox_inches='tight')

In [ ]:
for m in METHODS:
    vals = df.loc[df.method == m].Calc.to_numpy()
    print(f'{m}: ${np.mean(vals[vals<10])*100:.2f} \pm {np.std(vals[vals<10])*100:.2f}$')

In [ ]:
pts_labels = ['Calc']
data = []
for train_loc_model, metrics in calc_metrics.items():
    train_loc, model = train_loc_model.split('_')
    for test_loc, test_loc_metrics in metrics.items():
        for i in range(len(test_loc_metrics['Loss'])):
            test_loc_data = {l: test_loc_metrics[f'{l}_DICE'][i] for l in pts_labels}
            test_loc_data['train_location'] = train_loc
            test_loc_data['test_location'] = test_loc
            test_loc_data['model'] = model
            data.append(test_loc_data)
df = pd.DataFrame(data)
flattened_data = []
for i, row in df.iterrows():
    for j, l in enumerate(pts_labels):
        k = 'dice' # _hps' if j < 3 else 'distance_ca'
        flattened_data.append({
            'train_location': row.train_location, 
            'test_location': row.test_location, 
            'model': row.model,
            # 'method': row.method,
            k: row[l]
        })
df = pd.DataFrame(flattened_data)
df

p = pval(df,'heidelberg','federated','unet','unet')
print('Local UNet / Fed UNet: ', p)

p = pval(df,'heidelberg','kd','unet','unet')
print('Local UNet / KD UNet: ', p)

p = pval(df,'heidelberg','federated','swinunetr','swinunetr')
print('Local Swin / Fed Swin: ', p)

p = pval(df,'heidelberg','kd','swinunetr','swinunetr')
print('Local Swin / KD Swin: ', p)

p = pval(df,'federated','kd','swinunetr','swinunetr')
print('Fed Swin / KD Swin: ', p)

p = pval(df,'federated','kd','unet','swinunetr')
print('Fed UNet / KD Swin: ', p)

p = pval(df,'federated','kd','unet','unet')
print('Fed UNet / KD UNet: ', p)

p = pval(df,'federated','federated','unet','swinunetr')
print('Fed UNet / Fed Swin: ', p)